In [2]:
%load_ext autoreload
%autoreload
import tensorflow as tf
import numpy as np
import surreal_single, config
import functools, operator, copy
import tensorflow.contrib.slim as slim
from nets import resnet_v2
from nets.mobilenet import mobilenet_v2
from tensor_info2 import INPUT_TENSOR_INFO, OUTPUT_TENSOR_INFO
tf.reset_default_graph()
'''
for x in surreal.load():
    print(x)
    break
'''

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'\nfor x in surreal.load():\n    print(x)\n    break\n'

In [13]:
types = tuple(t['type'] for t in INPUT_TENSOR_INFO)
config.BATCH_SIZE = 10
input_tensors = tf.data.Dataset.from_generator(surreal_single.load, types) \
                               .batch(config.BATCH_SIZE) \
                               .prefetch(config.PREFETCH_SIZE) \
                               .make_one_shot_iterator() \
                               .get_next()
tensors = {}
for tensor, info in zip(input_tensors, INPUT_TENSOR_INFO):
    tensor.set_shape(info['shape'])
    tensors[info['name']] = tensor
print(tensors)

{'image': <tf.Tensor 'IteratorGetNext:0' shape=(10, 400, 400, 3) dtype=float32>, 'hm': <tf.Tensor 'IteratorGetNext:1' shape=(10, 400, 400, 16) dtype=float32>, 'so_x': <tf.Tensor 'IteratorGetNext:2' shape=(10, 400, 400, 16) dtype=float32>, 'so_y': <tf.Tensor 'IteratorGetNext:3' shape=(10, 400, 400, 16) dtype=float32>, 'mo_x': <tf.Tensor 'IteratorGetNext:4' shape=(10, 400, 400, 15) dtype=float32>, 'mo_y': <tf.Tensor 'IteratorGetNext:5' shape=(10, 400, 400, 15) dtype=float32>}


In [14]:
config.STRIDE = 16

MD_H = int(config.TAR_H/config.STRIDE)
MD_W = int(config.TAR_W/config.STRIDE)

DEPTH = [ti['shape'][-1] for ti in OUTPUT_TENSOR_INFO]
RESULT_SHAPE = (config.BATCH_SIZE, MD_H, MD_W, sum(DEPTH))
RESULT_SIZE = functools.reduce(operator.mul, RESULT_SHAPE[1:])
OUTPUT_SHAPE = (config.BATCH_SIZE, config.TAR_H, config.TAR_W, sum(DEPTH))
OUTPUT_SIZE = functools.reduce(operator.mul, OUTPUT_SHAPE[1:])


In [15]:
def bilinear(indices):
    oy = tf.clip_by_value(indices[1], 0, MD_H-1)
    ox = tf.clip_by_value(indices[2], 0, MD_W-1)
    iy = [tf.floor(oy), tf.ceil(o)]
    ix = [tf.floor(ox), tf.ceil(ox)]
    idx_p = []
    for y in iy:
        for x in ix:
            indices[1] = y
            indices[2] = x
            idx = tf.cast(tf.stack(indices, axis=-1), tf.int32)
            p = (1 - tf.abs(y - oy)) * (1 - tf.abs(x - ox))
            idx_p.append((idx, p))
    return idx_p

def gather_bilinear(params, indices):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.gather_nd(params, idx)
        res.append(r * p)
    return tf.add_n(res)

def scatter_bilinear(params, indices, shape):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.scatter_nd(idx, params, shape)
        if len(r.shape) > len(p.shape):
            p = tf.expand_dims(p, axis=-1)
        res.append(r * p)
    return tf.add_n(res)

In [16]:
def resize(tensor):
    return tf.image.resize_images(
        tensor,
        (config.TAR_H, config.TAR_W),
        method=tf.image.ResizeMethod.BICUBIC,
        align_corners=True)

with slim.arg_scope(resnet_v2.resnet_arg_scope()):
    model_output, _ = resnet_v2.resnet_v2_101(tensors['image'], output_stride=config.STRIDE)
    hm_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1], activation_fn=tf.sigmoid)
    so_x_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1], activation_fn=None)
    so_y_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1], activation_fn=None)
    mo_x_pred = slim.conv2d(model_output, config.NUM_EDGE, [1, 1], activation_fn=None)
    mo_y_pred = slim.conv2d(model_output, config.NUM_EDGE, [1, 1], activation_fn=None)

    b, y, x, i = np.mgrid[:config.BATCH_SIZE, :MD_H, :MD_W, :config.NUM_EDGE]
    for _ in range(config.NUM_RECURRENT):
        mo_p = [b, y+mo_y_pred, x+mo_x_pred, i]
        mo_x_pred = gather_bilinear(so_x_pred, mo_p) + mo_x_pred
        mo_y_pred = gather_bilinear(so_y_pred, mo_p) + mo_y_pred
    hm_out, so_x_out, so_y_out, mo_x_out, mo_y_out = [resize(x) for x in [hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred]]


In [20]:

saver = tf.train.Saver()
with tf.device('/cpu:0'):
    with tf.Session() as sess:
        checkpoint_path = 'logs/res_log_test/model.ckpt-672'
        saver.restore(sess, checkpoint_path)
        res = sess.run([tensors['image'], hm_out])


resnet_v2_101/conv1/weights : resnet_v2_101/conv1/weights:0
resnet_v2_101/conv1/biases : resnet_v2_101/conv1/biases:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0
resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights : resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0
resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/biases : resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/biases:0
resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/weights : resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/weights:0
resnet_v2_101/block1/unit_1/bottleneck

In [21]:
import time, os, shutil
#log_dir = 'logs/log_' + str(time.time())[-5:]
log_dir = 'logs/res_log_test'
shutil.rmtree(log_dir)
os.mkdir(log_dir)

In [ ]:
def InitAssignFn(sess):
    sess.run(init_assign_op, init_feed_dict)
tf.contrib.slim.learning.train(train_op,
                               '/home/ubuntu/personlab/'+log_dir,
                               init_fn=InitAssignFn,
                               log_every_n_steps=100,
                               save_summaries_secs=30,
                              )

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:Recording summary at step 3.
INFO:tensorflow:Recording summary at step 9.
INFO:tensorflow:Recording summary at step 14.
INFO:tensorflow:Recording summary at step 20.
INFO:tensorflow:Recording summary at step 25.
INFO:tensorflow:Recording summary at step 31.
INFO:tensorflow:Recording summary at step 37.
INFO:tensorflow:Recording summary at step 42.
INFO:tensorflow:Recording summary at step 48.
INFO:tensorflow:Recording summary at step 54.
INFO:tensorflow:Recording summary at step 59.
INFO:tensorflow:Recording summary at step 65.
INFO:tensorflow:Recording summary at step 70.
INFO:tensorflow:Recording summary at step 76.
INFO:tensorflow:Recording summary at step 81.
INFO:tensorflow:Recordi

In [ ]:
from tensorflow.python.tools import inspect_checkpoint
inspect_checkpoint.print_tensors_in_checkpoint_file(checkpoint_path, tensor_name='', all_tensors=False, all_tensor_names=True)

In [24]:
config.STRIDE = 16

MD_H = int(config.TAR_H/config.STRIDE)
MD_W = int(config.TAR_W/config.STRIDE)
def bilinear(indices):
    oy = indices[1] #tf.clip_by_value(indices[1], 0, MD_H-1)
    ox = indices[2] #tf.clip_by_value(indices[2], 0, MD_W-1)
    iy = [np.floor(oy), np.ceil(oy+1e-9)]
    ix = [np.floor(ox), np.ceil(ox+1e-9)]
    
    idx_p = []
    for y in iy:
        for x in ix:
            indices[1] = y
            indices[2] = x
            idx = np.stack(indices, axis=-1).astype(np.int32)
            p = (1 - np.abs(y - oy)) * (1 - np.abs(x - ox))
            idx_p.append((idx, p))
    return idx_p

bilinear(np.array([1.,2.01,3.0,4.]))

[(array([1, 2, 3, 4], dtype=int32), 0.9900000000000002),
 (array([1, 2, 4, 4], dtype=int32), 0.0),
 (array([1, 3, 3, 4], dtype=int32), 0.009999999999999787),
 (array([1, 3, 4, 4], dtype=int32), 0.0)]

In [23]:
set([1,1])

{1}